**ATENÇÃO: O código abaixo tem propósito explicativo do cenário, para a execução ele deverá ser executado localmente na máquina, a qual inclusive deve possuir as pastas referenciadas.**

In [ ]:
import os
import datetime
import shutil

*  Inicialmente, importar as bibliotecas, as quais serão utilizadas. Tais são
nativas do Python, facilitando na implantação e execução do ambiente.
 * A ‘os’ será utilizada para a manipulação dos diretórios;
 * A datetime traz a tipagens como timestamp e datetime para operar e usufruir de datas e horários;
 * E por último, a shutil será acionada quando for realizada as cópias, facilitando o processo.

In [ ]:
"""A função abaixo ela utiliza-se de metódos da biblioteca 'os' para extrair os metadados de um arquivo especificado através do caminho."""


def get_metadate(path_local):
    stts = os.stat(path_local)
    #O metódo stat executa uma extração de uma série de valores e retorna numa grande tupla
    name = os.path.basename(path_local)  #Extrai o nome do arquivo
    creation_data = datetime.datetime.fromtimestamp(stts.st_ctime)
    #Iremos armazenar esses valores convertendo para datetime
    modf_data = datetime.datetime.fromtimestamp(stts.st_mtime)
    return name, creation_data, modf_data, stts.st_size

* A função acima é autoexplicativa em finalidade, ela utiliza-se de métodos da biblioteca 'os' para extrair os metadados de um arquivo especificado através do caminho. Usar a função torna o código mais modular e traz manutenibilidade.
 * Tal caminho é informado através do parâmetro, ela recebe-o como variável local, logo tendo internamente o endereço do arquivo que será trabalhado;
 * Em seguida, o método stat nos dá quase tudo que queremos, ele executa uma extração de uma série de valores e retorna-os numa grande tupla;
 * Contudo, é necessário conversões, pois o stat traz informações de data em timestamp, então para melhor leitura e uso os valores serão convertidos para algo menos granular, um datetime.
 * A função retornará esses valores em uma 4-upla, incluindo também o tamanho vindo da variável stts, do método stat. Poderia ser armazenado esses dados em diversas estruturas como uma 'string' e concatenar para posteriormente dá apenas um write no arquivo criado, mas manipular 'string' é custoso e um único write também poderia criar um gargalo no processo a depender da quantia de arquivos. Outra opção seria criar uma lista como buffer, mas isso remeteria novamente a apenas um único write. Com uma tupla alcança-se um bom meio termo (um write para cada arquivo com todos os metadados solicitados).

In [ ]:
"""Definimos as variáveis alvo do processo, ou seja a origem, o destino e a raiz."""
root = 'home/valcann/'
#Caso queira testar localmente com os diretórios dados, talvez seja preciso substituir o valor por apenas 'home/valcann/
src = root + 'backupsFrom'
dst = root + 'backupsTo'
if not os.path.exists(dst):
    os.makedirs(dst)

* Para fins de desempenho pode optar-se por definir os caminhos como constantes concatenando com a raiz do diretório que estão ao invés de usar '.join' desnecessariamente;
* Será na raiz que o ‘log’ será registrado, a fim de haver auditabilidade desse backup;
* Caso não exista o diretório de destino, será criado.


In [ ]:
log_from = os.path.join(root, 'backupsFrom.log')
log_to = os.path.join(root, 'backupsTo.log')

limit_data = datetime.datetime.now() - datetime.timedelta(days=3)

* Mas para os arquivos, a fim de assegurar a formatação correta, ou seja que os caminhos fornecidos para estes tenham a terminação correta, com uma barra (/) por exemplo, evitando problemas de concatenação, opta-se por usar o '.join juntamente com a raiz;
* A constante ‘limit_data’ será definida antes do processo, sendo utilizada como critério para remoções e cópias.


In [ ]:
file_processed = 0
file_coppied = 0
file_removed = 0

* Esses três ‘int’ setados inicialmente em 0 tratam-se de contadores que irão ser utilizados para informar ao usuário ao fim do processo como tal foi concluído, trazendo um feedback.

In [ ]:
with open(log_from, 'w') as reg_from:  #1. Irei listar todos os arquivos detalhadamente que estão na origem.
    for path, directories, files in os.walk(src):
        """Retorna uma 3-pla com o primeiro valor sendo o caminho do arquivo, o segundo sendo os diretórios presentes e o
        último uma lista com os arquivos."""
        for f_current in files:  #Vamos explorar essa lista.
            absolute_path = os.path.join(path, f_current)
            destination_path = os.path.join(dst, f_current)
            metadate_file = get_metadate(absolute_path)  #Metadados.
            reg_from.write(f"Nome: {metadate_file[0]}, Tamanho: {metadate_file[3]} bytes, Criado em: {metadate_file[1]}, Modificado em: {metadate_file[2]}\n")
            file_processed += 1
            """2. Acima salvo no arquivo aberto que apesar doa lias como reg_from trata-se do recém-criado 'backupsFrom.log',
            faço escrita nele conforme os valores armazenados na tupla 'metadata_file' que contém o retorno da função."""
            if metadate_file[1] < limit_data:
                os.remove(absolute_path)  #3. Apesar de registrar irei remover os mais antigos (respeitando o limite de 3 dias dado).
                file_removed += 1
            else:  #4. Mas irei fazer cópias no destino apenas daqueles mais recentes.
                shutil.copy2(absolute_path, destination_path)
                file_coppied += 1
                with open(log_to, 'a') as reg_to:
                    """5. Por fim, na mesmo condicional (arquivos mais recentes), ou seja, tendo sido ele parte do backup,
                    irei registar isso em um 'log' para indicar que ele integrou esse processo de cópia."""
                    reg_to.write(f"Backup do arquivo feito em {datetime.datetime.now()}:\n"
                                 f"Nome: {metadate_file[0]}, Tamanho: {metadate_file[3]} bytes, "
                                 f"Criado em: {metadate_file[1]}, Modificado em: {metadate_file[2]}\n"
                                 f"Copiado para: {destination_path}, Original no: {absolute_path}\n" + "-" * 35 + "\n")

**1. Irá ser listado todos os arquivos detalhadamente que estão na origem neste bloco;**
* Utiliza-se 'with' para abrir o arquivo especificado (primeiro parâmetro) com maior segurança e no segundo parâmetro especifica-se o modo, no caso escrita (‘w’), pois irá salvar informações. Por fim, a manipulação será feita utilizando um alias ("reg_from") para o arquivo;
* O método walk da biblioteca 'os' explora similarmente a uma recursão um diretório recebido como parâmetro,  verificando os arquivos no caminho e em pastas mais internas. Retorna uma 3-pla com o primeiro valor sendo o caminho do arquivo, o segundo sendo os diretórios presentes e o último uma lista com os arquivos, essa será explorando para que cada arquivo seja coletado;
* A ‘absolute_path’ (como diz o nome) trata-se do caminho absoluto do arquivo que está sendo analisado;
* Antecipadamente cria-se o caminho onde deve ser copiado, o endereço final, de destino;
* Usa-se o join para que ser assegurado corretamente a concatenação aqui, incluindo a divisão usada pelo SO (como a '/');

**2. Então, salva-se no arquivo aberto, o qual apesar do alias como ‘reg_from’ trata-se do recém-criado 'backupsFrom.log', é realizada a escrita nele conforme os valores armazenados na tupla 'metadata_file' que contém o retorno da função;**

**3. Apesar de registrar, será removido os mais antigos (respeitando o limite de 3 dias dado).**

**4. Mas as cópias feitas no destino serão apenas daqueles arquivos mais recentes;**

* Isso é feito utilizando o método ‘copy2’, o qual além de preservar o payload preserva os metadados;
* Utiliza-se o 'a' (append), pois no estado 'w' iria ser sobrescrito o arquivo a cada iteração ao invés de adicionar novas informações;

**5. Por fim, na mesmo condicional (arquivos mais recentes), ou seja, tendo sido o arquivo parte do backup, será registrado em um 'log' para indicar que ele integrou esse processo de cópia.**

In [ ]:
print(f"Backup concluido:\n{file_processed} foram processados.\n"
      f"{file_removed} foram removidos conforme as configurações.\n"
      f"{file_coppied} foram copiados conforme as configurações.")

Backup concluido:
0 foram processados.
0 foram removidos conforme as configurações.
0 foram copiados conforme as configurações.


* Por fim, é apenas informado o sucesso do processo ao usuário juntamente com informações quantitativas e qualitativas.